Segmenting and Clustering Neighborhoods in Toronto

1. Assignment's description

In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

2. Scrap content from wiki page

Import necessary packages.

In [1]:
import pandas as pd
import requests
import numpy as np

!pip install beautifulsoup4
from bs4 import BeautifulSoup

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!pip install lxml
!pip install et_xmlfile

print('Folium installed')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

Scrape the "raw" table.

In [2]:

source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')

table = soup.find("table")
table_rows = table.tbody.find_all("tr")

res = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [tr.text for tr in td]
    
    # Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    if row != [] and row[1] != "Not assigned\n":
        # If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough.
        if "Not assigned\n" in row[2]: 
            row[2] = row[1]
        res.append(row)

# Dataframe with 3 columns
My_df = pd.DataFrame(res, columns = ["PostalCode", "Borough", "Neighborhood"])
My_df.head()

,PostalCode,Borough,Neighborhood
0,M3A\n,North York\n,Parkwoods\n
1,M4A\n,North York\n,Victoria Village\n
2,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
3,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n"
4,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n"


Remove "\n" at the end of each string in all the columns

In [3]:
My_df["PostalCode"] = My_df["PostalCode"].str.replace("\n","")
My_df["Borough"] = My_df["Borough"].str.replace("\n","")
My_df["Neighborhood"] = My_df["Neighborhood"].str.replace("\n","")
My_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Group all neighborhoods with the same postal code

In [4]:
My_df = My_df.groupby(["PostalCode", "Borough"])["Neighborhood"].apply(", ".join).reset_index()
My_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [5]:
print("Shape: ", My_df.shape)

Shape:  (103, 3)


3. Get the latitude and the longitude coordinates of each neighborhood.

We use the given csv file.

In [6]:
My_df_geo_coor = pd.read_csv("http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv")
My_df_geo_coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


We need to couple 2 dataframes "My_df" and "My_df_geo_coor" into one dataframe.

In [8]:
My_df_toronto = pd.merge(My_df, My_df_geo_coor, how='left', left_on = 'PostalCode', right_on = 'Postal Code')
# remove the "Postal Code" column
My_df_toronto.drop("Postal Code", axis=1, inplace=True)
My_df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


4. Explore and cluster the neighborhoods in Toronto

4.1. Get the latitude and longitude values of Toronto.

In [9]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


4.2. Create a map of the whole Toronto City with neighborhoods superimposed on top.

In [10]:
# create map of Toronto using latitude and longitude values
My_map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
My_map_toronto

Add markers to the map.

In [11]:
for lat, lng, borough, neighborhood in zip(
        My_df_toronto['Latitude'], 
        My_df_toronto['Longitude'], 
        My_df_toronto['Borough'], 
        My_df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(My_map_toronto)  

My_map_toronto

4.3. Map of a part of Toronto City

We are going to work with only the boroughs that contain the word "Toronto".

In [12]:
# "denc" = [D]owntown Toronto, [E]ast Toronto, [N]orth Toronto, [C]entral Toronto
My_df_toronto_denc = My_df_toronto[My_df_toronto['Borough'].str.contains("Toronto")].reset_index(drop=True)
My_df_toronto_denc.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Plot again the map and the markers for this region.

In [13]:
My_map_toronto_denc = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, borough, neighborhood in zip(
        My_df_toronto_denc['Latitude'], 
        My_df_toronto_denc['Longitude'], 
        My_df_toronto_denc['Borough'], 
        My_df_toronto_denc['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(My_map_toronto_denc)  

My_map_toronto_denc

4.4. Define Foursquare Credentials and Version

On the public repository on Github, I remove this field for the privacy!

In [14]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '' # Foursquare API version


4.5. Explore the first neighborhood in our data frame "My_df_toronto"

In [15]:
My_neighborhood_name = My_df_toronto_denc.loc[0, 'Neighborhood']
print(f"The first neighborhood's name is '{My_neighborhood_name}'.")

The first neighborhood's name is 'The Beaches'.


Get the neighborhood's latitude and longitude values.

In [16]:
neighborhood_latitude = My_df_toronto_denc.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = My_df_toronto_denc.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(My_neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


Now, let's get the top 100 venues that are in The Beaches within a radius of 500 meters.

In [17]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

# get the result to a json file
results = requests.get(url).json()

Function that extracts the category of the venue

In [18]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [19]:
My_venues = results['response']['groups'][0]['items']
My_nearby_venues = json_normalize(My_venues) # flatten JSON

# filter columns
My_filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
My_nearby_venues = My_nearby_venues.loc[:, My_filtered_columns]

# filter the category for each row
My_nearby_venues['venue.categories'] = My_nearby_venues.apply(get_category_type, axis=1)

# clean columns
My_nearby_venues.columns = [col.split(".")[-1] for col in My_nearby_venues.columns]

My_nearby_venues

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869
4,Seaspray Restaurant,Asian Restaurant,43.678888,-79.298167


4.6. Explore neighborhoods in a part of Toronto City

We are working on the data frame df_toronto_denc. Recall that, this region contain DENC of Toronto where,

"DENC" = [D]owntown Toronto, [E]ast Toronto, [N]orth Toronto, [C]entral Toronto

First, let's create a function to repeat the same process to all the neighborhoods in DENC of Toronto.

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    My_nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    My_nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(My_nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called toronto_denc_venues

In [21]:
My_toronto_denc_venues = getNearbyVenues(names=My_df_toronto_denc['Neighborhood'],
                                   latitudes=My_df_toronto_denc['Latitude'],
                                   longitudes=My_df_toronto_denc['Longitude']
                                  )

In [22]:
My_toronto_denc_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Beaches,43.676357,-79.293031,Seaspray Restaurant,43.678888,-79.298167,Asian Restaurant


Let's check how many venues were returned for each neighborhood.

In [23]:
My_toronto_denc_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",18,18,18,18,18,18
Central Bay Street,64,64,64,64,64,64
Christie,16,16,16,16,16,16
Church and Wellesley,75,75,75,75,75,75
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,35,35,35,35,35,35


Let's find out how many unique categories can be curated from all the returned venues

In [24]:
print('There are {} uniques categories.'.format(len(My_toronto_denc_venues['Venue Category'].unique())))

There are 231 uniques categories.


4.7. Analyze Each Neighborhood

In [25]:
# one hot encoding
My_toronto_denc_onehot = pd.get_dummies(My_toronto_denc_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
My_toronto_denc_onehot['Neighborhood'] = My_toronto_denc_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [My_toronto_denc_onehot.columns[-1]] + list(My_toronto_denc_onehot.columns[:-1])
My_toronto_denc_onehot = My_toronto_denc_onehot[fixed_columns]

My_toronto_denc_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [26]:
My_toronto_denc_grouped = My_toronto_denc_onehot.groupby('Neighborhood').mean().reset_index()
My_toronto_denc_grouped.head()

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.017241,0.0,0.0,0.000000,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
2,"Business reply mail Processing Centre, South C...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
4,Central Bay Street,0.015625,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.015625,0.0,0.0,0.015625,0.0


Check the 10 most common venues in each neighborhood.

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
My_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
My_neighborhoods_venues_sorted['Neighborhood'] = My_toronto_denc_grouped['Neighborhood']

for ind in np.arange(My_toronto_denc_grouped.shape[0]):
    My_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(My_toronto_denc_grouped.iloc[ind, :], num_top_venues)

My_neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Beer Bar,Farmers Market,Cocktail Bar,Bakery,Seafood Restaurant,Cheese Shop,Café,Restaurant,Irish Pub
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Convenience Store,Climbing Gym,Stadium,Bar,Italian Restaurant,Bakery,Restaurant
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Smoke Shop,Auto Workshop,Pizza Place,Comic Shop,Burrito Place,Restaurant,Brewery,Skate Park,Spa
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Harbor / Marina,Bar,Rental Car Location,Sculpture Garden,Plane,Coffee Shop
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Department Store,Bar,Japanese Restaurant,Salad Place,Bubble Tea Shop,Burger Joint


4.8. Cluster neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [28]:
# set number of clusters
kclusters = 5

My_toronto_denc_grouped_clustering = My_toronto_denc_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(My_toronto_denc_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [29]:
# add clustering labels
My_neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

My_toronto_denc_merged = My_df_toronto_denc

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
My_toronto_denc_merged = My_toronto_denc_merged.join(My_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

My_toronto_denc_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Asian Restaurant,Health Food Store,Pub,Trail,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Women's Store
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Restaurant,Furniture / Home Store,Pub,Indian Restaurant,Spa,Dessert Shop
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Pet Store,Fish & Chips Shop,Sandwich Place,Board Shop,Brewery,Restaurant,Italian Restaurant,Burrito Place,Ice Cream Shop,Steakhouse
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Gastropub,Bakery,Brewery,American Restaurant,Comfort Food Restaurant,Sandwich Place,Cheese Shop,Clothing Store
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Photography Studio,Bus Line,Park,Swim School,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


Finally, let's visualize the resulting clusters

In [30]:
# create map
My_map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        My_toronto_denc_merged['Latitude'], 
        My_toronto_denc_merged['Longitude'], 
        My_toronto_denc_merged['Neighborhood'], 
        My_toronto_denc_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(My_map_clusters)
       
My_map_clusters

4.9. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster.

Cluster 1

In [31]:
My_toronto_denc_merged.loc[My_toronto_denc_merged['Cluster Labels'] == 0, My_toronto_denc_merged.columns[[1] + list(range(5, My_toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Asian Restaurant,Health Food Store,Pub,Trail,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Women's Store
1,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Restaurant,Furniture / Home Store,Pub,Indian Restaurant,Spa,Dessert Shop
2,East Toronto,0,Pet Store,Fish & Chips Shop,Sandwich Place,Board Shop,Brewery,Restaurant,Italian Restaurant,Burrito Place,Ice Cream Shop,Steakhouse
3,East Toronto,0,Café,Coffee Shop,Gastropub,Bakery,Brewery,American Restaurant,Comfort Food Restaurant,Sandwich Place,Cheese Shop,Clothing Store
5,Central Toronto,0,Hotel,Park,Food & Drink Shop,Pizza Place,Breakfast Spot,Gym,Gym / Fitness Center,Department Store,Sandwich Place,Dessert Shop
6,Central Toronto,0,Coffee Shop,Clothing Store,Yoga Studio,Fast Food Restaurant,Chinese Restaurant,Cosmetics Shop,Café,Restaurant,Mexican Restaurant,Salon / Barbershop
7,Central Toronto,0,Pizza Place,Sandwich Place,Dessert Shop,Sushi Restaurant,Gym,Italian Restaurant,Café,Coffee Shop,Park,Brewery
9,Central Toronto,0,Coffee Shop,Pub,Sushi Restaurant,Fried Chicken Joint,Liquor Store,Restaurant,Bank,Sports Bar,Bagel Shop,Supermarket
11,Downtown Toronto,0,Coffee Shop,Restaurant,Pizza Place,Bakery,Pub,Market,Café,Chinese Restaurant,Italian Restaurant,Beer Store
12,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar,Yoga Studio,Pizza Place,Men's Store,Mediterranean Restaurant,Bubble Tea Shop


Cluster 2

In [32]:
My_toronto_denc_merged.loc[My_toronto_denc_merged['Cluster Labels'] == 1, My_toronto_denc_merged.columns[[1] + list(range(5, My_toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,1,Gym,Tennis Court,Women's Store,Cupcake Shop,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


Cluster 3

In [33]:
My_toronto_denc_merged.loc[My_toronto_denc_merged['Cluster Labels'] == 2, My_toronto_denc_merged.columns[[1] + list(range(5, My_toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,2,Pool,Garden,Women's Store,Cuban Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


Cluster 4

In [34]:
My_toronto_denc_merged.loc[My_toronto_denc_merged['Cluster Labels'] == 3, My_toronto_denc_merged.columns[[1] + list(range(5, My_toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,3,Jewelry Store,Trail,Mexican Restaurant,Sushi Restaurant,Women's Store,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


Cluster 5

In [35]:
My_toronto_denc_merged.loc[My_toronto_denc_merged['Cluster Labels'] == 4, My_toronto_denc_merged.columns[[1] + list(range(5, My_toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,4,Photography Studio,Bus Line,Park,Swim School,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
10,Downtown Toronto,4,Park,Playground,Trail,Women's Store,Cuban Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
